# Courses recommendation system
# I. Extract, transform and load

This is the first part of the Udacity Data Science Nanodegree capstone project, which consists in the creation of a course recommendation system.

## Introduction

[Emagister](https://www.emagister.com) is a company whose objective is to be a meeting point for students and course providers and they aim to do so by helping people find the right training. That's why the recommender system is one of the most important parts of the web. So the main motivation of this project is to improve the current recommendation system.

### Data used for the project

The data that will be worked on in this project are real data extracted from the [Emagister UK](https://www.emagister.co.uk) database. As an employee of Emagister, I requested authorization from the company to use the data, after consulting with our lawyers, the company permitted me. 

For security and legal reasons, user data is encrypted. 

### The recommender system

The recommendations are based on the following four methods of recommendations:

* Knowledge based recommendations
* Content based filtering
* Neighborhood based collaborative filtering
* Model based collaborative filtering

The project is divided in five parts:

1. ETL pipeline
2. Exploratory data analysis
3. Models creation
4. Make recommendations
5. A demo web application

#### ETL pipeline

The pipeline retrieves raw data from database, then performs the data wrangling process on this data and finally loads the resulting clean data to database and files, ready to be used in the web application.

I have taken the code written in this section and arranged into several classes and a script, which allows you to automate the ETL process.


#### Exploratory data analysis

Once the data is cleaned, is time to perform an exploratory data analysis. I will search for patterns and trends in data and I will create visualizations for this data as well.

#### Models creation

In this part, I will create models from which I will make the recommendations.

#### Make recommendations

After the exploratory data analysis, is time to play around with structures created in the first part and trying to make recommendations.

#### Demo web

The observations and models derived from the ETL phase and analysis of the project, have been put into practice in a web application that can be accessed [here](https://courses-recommender.herokuapp.com/).

## Dependencies
To run this project properly, you need the following:

* Python >=3.5
* numpy 1.18.1
* pandas 0.24.2
* scikit-learn 0.20.3
* scipy 1.2.1
* sqlalchemy 1.3.2
* matplotlib 3.0.3
* halo 0.0.28 (Spinner for terminal. [PyPi](https://pypi.org/project/halo/))
* pymysql 0.9.3 (Python MySQL client library. [PyPi](https://pypi.org/project/PyMySQL/))

In addition you need to install `texcptulz`, a library designed to transform the raw ingested text into a form that is ready for calculation and modelling.
This library has been developed by me for this project. To install `texcptulz` run the following command:

`pip install texcptulz`

More information [here](https://pypi.org/project/texcptulz/).

# Start ETL Process
## 1 Import libraries

In [138]:
import numpy as np
import pandas as pd

from txtools.normalizer import clean_text
from txtools.utils import LangDetector
from db_utils import connection


## 2 Retrieving Leads
I will get a sample of leads from Emagister database gererated in the last year. The sample table `leads` contains approximately 40000 leads.

In [68]:
leads_query = '''SELECT user_id,
    course_id,
    course_title,
    course_description,
    course_category,
    center,
    created_on
FROM leads
ORDER BY created_on DESC
'''

leads_df = pd.read_sql_query(leads_query, con=connection())

In [69]:
leads_df.head()

,user_id,course_id,course_title,course_description,course_category,center,created_on
0,ed53e691ee322e24d8cc843fff68ebc6,170631539,Aviation Engineering - BEng (Hons),The Aviation Engineering programme aims to tra...,Engineering,The Hong Kong Polytechnic University,2020-01-05 13:13:03
1,ed53e691ee322e24d8cc843fff68ebc6,170631539,Aviation Engineering - BEng (Hons),The Aviation Engineering programme aims to tra...,Engineering,The Hong Kong Polytechnic University,2020-01-05 10:59:56
2,ed53e691ee322e24d8cc843fff68ebc6,170631539,Aviation Engineering - BEng (Hons),The Aviation Engineering programme aims to tra...,Engineering,The Hong Kong Polytechnic University,2020-01-04 21:10:52
3,ed53e691ee322e24d8cc843fff68ebc6,170609773,Master of Leadership in Development Finance - ...,Boost your career with an international degree...,Accounting,Frankfurt School of Finance & Management,2020-01-03 20:22:46
4,ed53e691ee322e24d8cc843fff68ebc6,170654780,New to Web Design,Our New to Web Design course bundle will teach...,Other Web Design,IT Online Learning,2020-01-01 20:29:13


### 2.1 Search for duplicated
Duplicated leads, in this case is when a user generates more than one lead into the same course

In [70]:
leads_df.duplicated(['user_id', 'course_id']).sum()

255

I will remove the duplicates keeping the first lead generated by the user, that's the one that was created earlier.

In [71]:
leads_df.drop_duplicates(['user_id', 'course_id'], inplace=True, keep='last')

In [72]:
leads_df.duplicated(['user_id', 'course_id']).sum()

0

### 2.2 Search for missing values

In [73]:
leads_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39782 entries, 4 to 40036
Data columns (total 7 columns):
user_id               39782 non-null object
course_id             39782 non-null int64
course_title          39782 non-null object
course_description    38253 non-null object
course_category       39782 non-null object
center                39782 non-null object
created_on            39782 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 2.4+ MB


There are 1529 leads to courses without descriptions

In [74]:
leads_df[leads_df['course_description'].isnull()]['course_id'].nunique()

653

Or 653 courses without description. I will keep this rows in the dataframe because the information provided by a lead-course pair is very valuable even if the course has no description. Anyway, I will replace the null value with an empty string

In [75]:
leads_df['course_description'] = leads_df['course_description'].fillna('')

In [77]:
leads_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39782 entries, 4 to 40036
Data columns (total 7 columns):
user_id               39782 non-null object
course_id             39782 non-null int64
course_title          39782 non-null object
course_description    39782 non-null object
course_category       39782 non-null object
center                39782 non-null object
created_on            39782 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 2.4+ MB


### 2.3 Convert `course_id` column type to string

In [80]:
leads_df['course_id'] = leads_df['course_id'].astype(str)

In [81]:
leads_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39782 entries, 4 to 40036
Data columns (total 7 columns):
user_id               39782 non-null object
course_id             39782 non-null object
course_title          39782 non-null object
course_description    39782 non-null object
course_category       39782 non-null object
center                39782 non-null object
created_on            39782 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 2.4+ MB


## 3 Retrieving reviews
The reviews come from the Emagister database and are a sample of those that have been made in the last 18 months.

In [82]:
reviews_query = '''SELECT user_id,
    course_id,
    course_title,
    course_category,
    course_description,
    center,
    rating,
    created_on
FROM reviews
ORDER BY created_on DESC
'''

reviews_df = pd.read_sql_query(reviews_query, con=connection())

In [84]:
reviews_df.head()

,user_id,course_id,course_title,course_category,course_description,center,rating,created_on
0,2b8d8517932c25c11858c48b7563d1f0,170378812,SIA Bodyguard Training,Self-development,The SIA approved Close Protection course is th...,Clinton Training Ltd,6,2019-12-26 17:40:22
1,6c4962141ae920e56739fd36a1383333,170363242,NVQ Tiling courses - Free - Funded by Government,Construction Trades,"CSCS Skill Card. <br />Suitable for: Employed,...",Censura Consulting,10,2019-12-26 17:18:13
2,52d477c4ebcb10e3368a2fa396d97df7,170628943,Bachelor in Aviation Management,Energy and Utilities,Immerse yourself in the world of aviation mana...,IUBH University of Applied Sciences,10,2019-12-26 14:51:26
3,8921583e8a24ca92d0ae4b65d86d6c6a,170385895,Forensics,Police,Forensic science is a very exciting area as sh...,International Career Institute,10,2019-12-26 11:27:27
4,6c4962141ae920e56739fd36a1383333,170363242,NVQ Tiling courses - Free - Funded by Government,Construction Trades,"CSCS Skill Card. <br />Suitable for: Employed,...",Censura Consulting,10,2019-12-26 11:25:08


### 3.1 Search for outliers in `rating` column

In [85]:
reviews_df['rating'].describe()

count    19160.000000
mean         9.301461
std          1.142509
min          2.000000
25%          8.000000
50%         10.000000
75%         10.000000
max         10.000000
Name: rating, dtype: float64

There seems to be no outliers

### 3.2 Search for duplicated rows

In [86]:
reviews_df.duplicated(['user_id', 'course_id']).sum()

185

There are 185 duplicated reviews. I will remove them keeping the first review made by the user.

In [87]:
reviews_df.drop_duplicates(['user_id', 'course_id'], inplace=True, keep='last')

In [88]:
reviews_df.duplicated(['user_id', 'course_id']).sum()

0

### 3.3 Search for missing values

In [89]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18975 entries, 0 to 19159
Data columns (total 8 columns):
user_id               18975 non-null object
course_id             18975 non-null int64
course_title          18975 non-null object
course_category       18975 non-null object
course_description    18919 non-null object
center                18975 non-null object
rating                18975 non-null int64
created_on            18975 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 1.3+ MB


There are 56 reviews to courses without description.

In [90]:
reviews_df[reviews_df['course_description'].isnull()]['course_id'].nunique()

23

Or 23 courses without description. I will do the same as in the previous case. I will keep the rows to keep the information of each review. I will replace the null value with an empty string in this dataframe too.

In [99]:
reviews_df['course_description'] = reviews_df['course_description'].fillna('')

In [100]:
reviews_df[reviews_df['course_description'].isnull()]['course_id'].nunique()

0

### 3.4 Convert `course_id` column type to string

In [101]:
reviews_df['course_id'] = reviews_df['course_id'].astype(str)

In [102]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18975 entries, 0 to 19159
Data columns (total 8 columns):
user_id               18975 non-null object
course_id             18975 non-null object
course_title          18975 non-null object
course_category       18975 non-null object
course_description    18975 non-null object
center                18975 non-null object
rating                18975 non-null int64
created_on            18975 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.3+ MB


In [103]:
reviews_df.head()

,user_id,course_id,course_title,course_category,course_description,center,rating,created_on
0,2b8d8517932c25c11858c48b7563d1f0,170378812,SIA Bodyguard Training,Self-development,The SIA approved Close Protection course is th...,Clinton Training Ltd,6,2019-12-26 17:40:22
2,52d477c4ebcb10e3368a2fa396d97df7,170628943,Bachelor in Aviation Management,Energy and Utilities,Immerse yourself in the world of aviation mana...,IUBH University of Applied Sciences,10,2019-12-26 14:51:26
3,8921583e8a24ca92d0ae4b65d86d6c6a,170385895,Forensics,Police,Forensic science is a very exciting area as sh...,International Career Institute,10,2019-12-26 11:27:27
4,6c4962141ae920e56739fd36a1383333,170363242,NVQ Tiling courses - Free - Funded by Government,Construction Trades,"CSCS Skill Card. <br />Suitable for: Employed,...",Censura Consulting,10,2019-12-26 11:25:08
5,6c4962141ae920e56739fd36a1383333,170567406,Level 2 Diploma in Wall and Floor Tiling,Construction Trades,Level 2 Diploma in Wall and Floor Tiling\n« Re...,Hackney Community College,10,2019-12-26 11:19:59


## 4 Create courses dataframe

I will create a dataframe of unique courses from the reviews and leads dataframes

In [104]:
keep_columns = ['course_id', 'course_title', 'course_description', 'course_category', 'center']

courses_from_leads = leads_df[keep_columns].drop_duplicates('course_id')
courses_from_reviews = reviews_df[keep_columns].drop_duplicates('course_id')

courses_df = pd.merge(courses_from_leads, courses_from_reviews,
                      left_on=keep_columns, 
                      right_on=keep_columns, 
                      how='outer')

# Remove duplicates
courses_df.drop_duplicates('course_id', inplace=True)

courses_df.head()

,course_id,course_title,course_description,course_category,center
0,170654780,New to Web Design,Our New to Web Design course bundle will teach...,Other Web Design,IT Online Learning
1,170565168,Wordpress Website & Blog Builder,"Right now, there are about 6.7 million blogs o...",Other Web Design,International Open Academy
2,170079653,Office Skills Diploma,If you want to gain practical skills in order ...,Office Systems,Pitman Training London
3,170623853,Web Design Bundle Course - CPD Certified & IAO...,Web design is not only about planning and desi...,HomeSite,John Academy
4,170418482,Webmaster HTML & CSS Web Design Course,Emagister presents the Webmaster course - HTML...,Other Web Design,Distance Learning Centre


In [105]:
courses_df['course_id'].nunique()

17527

In [106]:
courses_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17527 entries, 0 to 17534
Data columns (total 5 columns):
course_id             17527 non-null object
course_title          17527 non-null object
course_description    17527 non-null object
course_category       17527 non-null object
center                17527 non-null object
dtypes: object(5)
memory usage: 821.6+ KB


Renaming the columns

In [107]:
courses_df.rename(columns={'course_id': 'id', 'course_title': 'title',
                          'course_description': 'description', 'course_category': 'category'},
                 inplace=True)

Cleaning text columns

In [108]:
courses_df['title'] = courses_df['title'].apply(clean_text)
courses_df['description'] = courses_df['description'].apply(lambda x: clean_text(x) if x else x)
courses_df['center'] = courses_df['center'].apply(clean_text)

### 4.1 Content language

Sometimes the course descriptions are not in the correct language. I will detect the language, and I will discard the courses with the description in a language other than English. <span style="color:red">**This process takes about 45 minutes on a MacBook Pro (2.3 GHz Intel Core i5, 16 GB RAM)**</span>

In [109]:
lang_detector = LangDetector()
def get_lang(text):
    """ 
    Detects the language of a text
    
    :param text str: Text of which we want to know the language
                          
    :return str: Language ISO 639-1 code
    """
    if not text:
        return LangDetector.DEFAULT_LANGUAGE
    
    try: 
        return lang_detector.iso_639_1_code(text)
    except ValueError:
        return LangDetector.DEFAULT_LANGUAGE

In [110]:
courses_df['lang'] = courses_df['description'].apply(get_lang)

In [111]:
not_english_courses = courses_df[courses_df['lang'] != 'en']
not_english_courses.head()

,id,title,description,category,center,lang
56,170573636,Criminologa,El curso est dirigido a todos aquellos que pro...,Criminal Law,Deka Online Courses,es
66,170661935,Business Model Innovation,Design Thinking parte de la idea de que la mej...,Business Management,Design Thinking Sweden,es
78,170232997,Introduction to Basic TIG Welding,To get basic skills in TIG welding. Suitable f...,Engineering,Allister Moore Weld Training Centre,xx
89,170573635,Educador Cinfilo,El curso est dirigido a todos aquellos que pro...,Animal Health,Deka Online Courses,es
120,170623758,Masaje y Masoterapia,El curso est dirigido a todos aquellos que pro...,Medicine,Deka Online Courses,es


Remove not english courses and column `lang`.

In [112]:
courses_df.drop(courses_df[courses_df['lang'] != 'en'].index, inplace=True)
courses_df.drop('lang', axis=1, inplace=True)

Remove courses written in a non-English language from `reviews_df` and `leads_df`

In [113]:
reviews_df = reviews_df[~reviews_df['course_id'].isin(not_english_courses['id'].values)]
leads_df = leads_df[~leads_df['course_id'].isin(not_english_courses['id'].values)]

### 4.2 Save leads and reviews to database
**Save leads**

In [135]:
# Creates the 'clean_leads' table

leads_df = leads_df[['user_id', 'course_id', 'created_on']]

sql_drop = 'DROP TABLE IF EXISTS `clean_leads`'
c = connection().execute(sql_drop)

sql_create = """CREATE TABLE clean_leads
(
    `user_id`    CHAR(36) NOT NULL,
    `course_id`  VARCHAR(12) NOT NULL,
    `created_on` DATETIME NOT NULL,
    PRIMARY KEY (`user_id`, `course_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
"""
c = connection().execute(sql_create)

# Save leads to database
leads_df.to_sql('clean_leads', con=connection(), if_exists='append', index=False)

**Save reviews**

In [136]:
# Creates the 'clean_reviews' table

reviews_df = reviews_df[['user_id', 'course_id', 'rating', 'created_on']]

sql_drop = 'DROP TABLE IF EXISTS `clean_reviews`'
c = connection().execute(sql_drop)

sql_create = """CREATE TABLE `clean_reviews`
(
    `user_id`    CHAR(36) NOT NULL,
    `course_id`  VARCHAR(12) NOT NULL,
    `rating`     INT NOT NULL,
    `created_on` DATETIME NOT NULL,
    PRIMARY KEY (`user_id`, `course_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
"""
c = connection().execute(sql_create)

# Save reviews to database
reviews_df.to_sql('clean_reviews', con=connection(), if_exists='append', index=False)


### 4.2 Computing the weighted rating

I will add a column to the courses_df indicating the weighted rating for each course. I will also add a column indicating the number of reviews of each course.

To compute the weighted rating, I will use the IMDB's weighted rating formula to calculate the [Top Rated 250 titles](https://help.imdb.com/article/imdb/track-movies-tv/ratings-faq/G67Y87TFYYP6TWAV#calculatetop):

$$
weighted\space rating\space(wr) = (\dfrac{v}{v+m}\times{R})+(\dfrac{m}{v+m}\times{C})
$$

Where:

- $v$ is the number of reviews received by the course
- $m$ is the minimum number of reviews required for the course to be listed in the top
- $R$ is the average rating of the course
- $C$ is the average rating of all the courses in the dataset

I can compute $v$, $R$ and $C$, but $m$ is a number that does not depend on any calculation. In IMDB, this number is 25,000. In Emagister, we award a prize to the most valued study centers and schools. One of the conditions to qualify for the prize is that each course has at least 25 reviews, so I will use 25 as a value for $m$.

With this calculation, I will take into account the number of reviews each course has received to rank courses.

In [114]:
def average_rating(course_id, df):
    """ 
    Computes the average rating of a course
    
    :param course_id str: Course id
                          
    :return float|np.nan: The average rating of a course or np.nan if the course has no rating
    """
    df = df[df['course_id'] == course_id]
    
    if df.shape[0] == 0:
        return np.nan
    
    return df['rating'].mean()


def num_reviews(course_id, df):
    """ 
    Counts the number of reviews of a course
    
    :param course_id str: Course id
                          
    :return int: The number of reviews of a course
    """
    df = df[df['course_id'] == course_id]
    
    return df.shape[0]


def all_avg_rating(df):
    """ 
    Computes the average rating of all courses (C)
    
    :param df DataFrame: Courses dataframe
                          
    :return float: The average rating of all courses in DataFrame
    """
    df = df[~df['avg_rating'].isnull()]
        
    return df['avg_rating'].mean()


def weighted_rating(row, C, m):
    """ 
    Computes the weighted rating of a course
    
    :param row DataFrame row: A DataFrame row representing a course
    :param C float: The average rating of all the courses in the dataset
    :param m int: The minimum number of reviews required for the course to be listed
                          
    :return float: The average rating of all courses in DataFrame
    """
    v = row['num_reviews']
    R = row['avg_rating']
    
    if v == 0 and np.isnan(R):
        return np.nan
    
    wr = (v * R / (v + m)) + (m * C / (v + m))
    
    return wr

I must first calculate the rating average of each course:

In [115]:
courses_df['avg_rating'] = courses_df['id'].apply(average_rating, args=(reviews_df,))

Then, I count the number of reviews received for each course:

In [116]:
courses_df['num_reviews'] = courses_df['id'].apply(num_reviews, args=(reviews_df,)).astype(int)

And finally, I will calculate the weighted rating. Using 25 as a value for $m$, as I explained above.

In [117]:
C = all_avg_rating(courses_df)
m = 25


courses_df['weighted_rating'] = courses_df.apply(weighted_rating, axis=1, args=(C, m,))

In [118]:
courses_df.sort_values('weighted_rating', ascending=False).head()

,id,title,description,category,center,avg_rating,num_reviews,weighted_rating
14544,170575432,Aerospace Engineering MEng (Hons) 4 years,The first two years of this course give you a ...,Engineering,University of Bath,10.0,44,9.731074
2949,170601662,Excel Intermediate Course,Course summary This Excel course is offered on...,Excel,PCWorkshops,10.0,44,9.731074
13730,170644676,Corporate English Training,Corporate English Training The London School o...,English,The London School of English,10.0,29,9.656372
11903,170040286,Drafting and Negotiating International Agency ...,This two-day interactive course is targeted a...,International Law,Falconbury Ltd,10.0,27,9.643155
11900,170040323,Drafting Commercial Contracts,Do you want to negotiate and draft clear and c...,Business Law,Falconbury Ltd,10.0,26,9.636158


### 4.3 Add number of leads to courses_df
Now, I will add a column to the courses_df indicating the number of leads generated for each course.

In [119]:
lead_counts = leads_df.groupby('course_id').count()['user_id']
def number_of_leads(course_id):
    """ 
    Counts the number of leads generated by a course
    
    :param course_id str: Course id
                          
    :return int: The number of leads generated by a course
    """
    try:
        return lead_counts.loc[course_id]
    except KeyError:
        return 0

In [120]:
courses_df['number_of_leads'] = courses_df['id'].apply(number_of_leads).astype(int)

In [121]:
courses_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17323 entries, 0 to 17534
Data columns (total 9 columns):
id                 17323 non-null object
title              17323 non-null object
description        17323 non-null object
category           17323 non-null object
center             17323 non-null object
avg_rating         7045 non-null float64
num_reviews        17323 non-null int64
weighted_rating    7045 non-null float64
number_of_leads    17323 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 1.3+ MB


### 4.4 Save courses to database

In [134]:
# Creates the 'courses' table

sql_drop = 'DROP TABLE IF EXISTS `courses`'
c = connection().execute(sql_drop)

sql_create = """CREATE TABLE `courses` (
  `id` varchar(9) NOT NULL,
  `title` text,
  `description` text,
  `center` varchar(100) NOT NULL,
  `avg_rating` double DEFAULT NULL,
  `num_reviews` int(11) DEFAULT NULL,
  `weighted_rating` double DEFAULT NULL,
  `number_of_leads` int(11) DEFAULT NULL,
  `category_id` int(11) NOT NULL,
  PRIMARY KEY (`id`),
  KEY `courses_category_id_index` (`category_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
"""
c = connection().execute(sql_create)

# Save courses to database
courses_df.to_sql('courses', con=connection(), if_exists='append', index=False)

## 5 Create a categories dataframe

I will create a categories DataFrame to store the categories of the courses. This will be useful in the web application to navigate through the different categories. 

In [122]:
categories = courses_df['category'].unique()

In [123]:
categories_df = pd.DataFrame(categories, columns=['name']).reset_index()

In [124]:
categories_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 2 columns):
index    253 non-null int64
name     253 non-null object
dtypes: int64(1), object(1)
memory usage: 4.0+ KB


In [125]:
# Convert the index into a column
categories_df.rename(columns={'index': 'id'}, inplace=True)

In [126]:
categories_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 2 columns):
id      253 non-null int64
name    253 non-null object
dtypes: int64(1), object(1)
memory usage: 4.0+ KB


In [127]:
# This is to avoid identifiers with 0 value
categories_df['id'] = categories_df['id'].apply(lambda x: x + 1)

In [128]:
categories_df.head()

,id,name
0,1,Other Web Design
1,2,Office Systems
2,3,HomeSite
3,4,Engineering
4,5,Marketing Operations


### 5.1 Add category id to courses dataframe

In [129]:
def category_id(category_name):
    """Search for a category by name in the category data frame and return its identifier

    :param category_name: Category name
    :return: The category id
    """
    return categories_df[categories_df['name'] == category_name]['id'].values[0]
    

In [130]:
courses_df['category_id'] = courses_df['category'].apply(category_id)

In [131]:
# Removes the category name from courses_df
courses_df.drop('category', inplace=True, axis=1)

### 5.2 Save categories to database

In [133]:
# Creates the 'categories' table

sql_drop = 'DROP TABLE IF EXISTS `categories`'
c = connection().execute(sql_drop)

sql_create = """CREATE TABLE `categories` (
  `id` int(11) NOT NULL,
  `name` varchar(200) NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8
"""
c = connection().execute(sql_create)

# Save categories to database
categories_df.to_sql('categories', con=connection(), if_exists='append', index=False)
